<a href="https://colab.research.google.com/github/gitconnoisseur/toxicity-model/blob/master/ecfp_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install --upgrade mxnet
#!pip install autogluon
!pip install PyTDC
!pip install deepchem
!pip install autokeras

     |████████████████████████████████| 87 kB 2.4 MB/s 
  Created wheel for PyTDC: filename=PyTDC-0.3.4-py3-none-any.whl size=119015 sha256=3395d61a84147995259b4e15e9bf4e48ef877afa3c549662d3c32d61bd8b1f2f
  Stored in directory: /root/.cache/pip/wheels/49/a8/46/5a1503adcd1143d3ade58f9888ee8da52dd7c8900e6bfb2a63
Successfully built PyTDC
     |████████████████████████████████| 608 kB 4.3 MB/s 
     |████████████████████████████████| 20.6 MB 1.7 MB/s 
     |████████████████████████████████| 15.7 MB 40.6 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium

     |████████████████████████████████| 166 kB 4.3 MB/s 
     |████████████████████████████████| 454.4 MB 25 kB/s 
     |████████████████████████████████| 97 kB 7.1 MB/s 
     |████████████████████████████████| 4.0 MB 29.6 MB/s 
     |████████████████████████████████| 462 kB 48.5 MB/s 
     |████████████████████████████████| 14.8 MB 1.1 MB/s 
     |████████████████████████████████| 1.2 MB 49.1 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68719 sha256=f79d4a2e4930819a3c55fea893d0f4e1a8d3a49b478b19045b8feb2306b05297
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1

In [ ]:
!pip install --upgrade numpy

  Using cached numpy-1.21.5-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.5 which is incompatible.
tensorflow 2.5.2 requires numpy~=1.19.2, but you have numpy 1.21.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
#import autogluon as ag
from tdc.single_pred import Tox
import deepchem

import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak

from tdc.benchmark_group import admet_group
group = admet_group(path = 'data/')

100%|██████████| 1.47M/1.47M [00:01<00:00, 1.34MiB/s]
Extracting zip file...
Done!


In [ ]:
predictions_list = []

for seed in [1, 2, 3, 4, 5]:
    benchmark = group.get('LD50_Zhu') 
    # all benchmark names in a benchmark group are stored in group.dataset_names
    predictions = {}
    name = benchmark['name']
    train_val, test = benchmark['train_val'], benchmark['test']
    #train, valid = group.get_train_valid_split(benchmark = name, split_type = 'default', seed = seed)
    
        # --------------------------------------------- # 
        #  Train your model using train, valid, test    #
        #  Save test prediction in y_pred_test variable #
        # --------------------------------------------- #
    

    featurizer = deepchem.feat.CircularFingerprint(size=2048, radius=4)
    f = featurizer.featurize(train_val.iloc[:,1].to_list())
    train_x = pd.DataFrame(f)
    train_y = train_val.iloc[:,2]

    reg = ak.StructuredDataRegressor(overwrite=True, max_trials=50, metrics=['mae'])
    reg.fit(x=train_x, y=train_y)

    testFeatures = pd.DataFrame(featurizer.featurize(test.iloc[:,1].to_list()))

    y_pred_test = reg.predict(testFeatures)

    print("predictions:")
    print(seed)
    print(y_pred_test[0:10])
        
    predictions[name] = y_pred_test
    predictions_list.append(predictions)

print("Prediction List: ")
print(predictions_list)

Trial 1 Complete [01h 10m 22s]
val_loss: 0.878301739692688

Best val_loss So Far: 0.878301739692688
Total elapsed time: 01h 10m 22s

Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
structured_data...|True              |True              
structured_data...|False             |False             
structured_data...|2                 |2                 
structured_data...|32                |32                
structured_data...|0                 |0                 
structured_data...|32                |32                
regression_head...|0                 |0                 
optimizer         |adam              |adam              
learning_rate     |0.0001            |0.001             

Epoch 1/1000
148/148 [==============================] - 90s 425ms/step - loss: 3.7320 - mae: 1.5836 - val_loss: 3.2309 - val_mae: 1.4545
Epoch 2/1000
148/148 [==============================] - 48s 326ms/step - loss: 1.4141 - mae: 0.9016 - val_loss: 2.0845 - val_mae: 1.1